In [1]:
import rdflib
import morph_kgc

# Construction, enrichment and evaluation of the Solarchem Knowledge Graph

## Mapping generation
Running Mapeathor to create the mapping file in YARRRML

In [3]:
!python -m mapeathor -i mappings/solarchem-mapping.xlsx -l yarrrml -o mappings/solarchem-mapping.yml

Generating mapping file
Your mapping file is in mappings/solarchem-mapping.yml


## KG construction
Running Morph-KGC to construct the KG with the mapping generated in the previous step and the data in `./data`.

In [4]:
!python -m morph_kgc config.ini

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
INFO | 2024-03-27 17:27:22,869 | 98 mapping rules retrieved.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rml_df = self.rml_df.replace(r'^\s*$', np.nan, regex=True)
INFO | 2024-03-27 17:27:23,027 | Mapping partition with 86 groups generated.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_partitioner.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a 

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

## KG validation
Quick validation to check that the KG is being generated with all the needed properties of the ontology.

In [5]:
g = rdflib.Graph()
g.parse("result/solarchem.nt")
print(len(g))

386347


### Articles

In [7]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article .
    }

""")

for row in q_res:
    print(f"Number of articles: {row['count'].value}")

Number of articles: 1268


In [8]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            bibo:doi ?doi ;
            bibo:abstract ?abs ;
            bibo:volume ?volume ;
            dc:title ?title ;
            bibo:issue ?issue ;
            dc:date ?date ;
            bibo:pageStart ?page .
    }

""")

for row in q_res:
    print(f"Number of articles with all properties: {row['count'].value}")

Number of articles with all properties: 415


In [9]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?article) AS ?count)
    WHERE {
        ?article a bibo:Article ;
            solar:hasExperimentExecution ?expexec .
    }

""")

for row in q_res:
    print(f"Number of articles with experiments associated: {row['count'].value}")

Number of articles with experiments associated: 1096


### Experiment Executions

In [10]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>

    SELECT (COUNT (DISTINCT ?input) AS ?count)
    WHERE {
        ?expexec a solar:ExperimentExecution ;
            prov:used ?input ;
            solar:hasCondition ?condition ;
            solar:hasLightSource ?lightsource .
    }

""")

for row in q_res:
    print(f"Number of experiments: {row['count'].value}")

Number of experiments: 18315


### Conditions

In [11]:
## Time, pressure, reactor, pH, temperature
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT ?condition ?value ?unit
    WHERE {
        ?condition a solar:TimeCondition ; # solarpc:ReactorCondition solar:PressureCondition solar:pHCondition solar:TemperatureCondition
            qudt:numericValue ?value ;
            #solarpc:reactorVolume ?value ;
            qudt:unit ?unit .
    } LIMIT 10
""")

for row in q_res:
    print(f"{row.condition} {row.value} {row.unit} ")

http://w3id.org/solar/i/Condition/Time/6153 5.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/2647 4.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/7645 1.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/2206 8.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/4608 4.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/5549 8.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/7077 4.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/5630 12.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/5820 1.0 http://qudt.org/vocab/unit/HR 
http://w3id.org/solar/i/Condition/Time/5212 10.0 http://qudt.org/vocab/unit/HR 


In [12]:
## Reactor medium and operation mode
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT *
    WHERE {
        ?expexec solar:hasCondition solar:GasMedium ; # solar:GasMedium
            solar:hasCondition solar:BatchMode . # solar:ContinuousMode
    } LIMIT 10
""")

for row in q_res:
    print(row)

(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/2228'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/3107'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/4630'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/2750'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/7364'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/4808'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/4727'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/4760'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/7704'),)
(rdflib.term.URIRef('http://w3id.org/solar/i/ExpExec/3927'),)


In [13]:
## Reactor medium and operation mode
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX obi: <http://purl.obolibrary.org/obo/OBI_>

    SELECT DISTINCT ?lightSource ?lamp ?wl ?irradiance ?power 
    WHERE {
        ?lightSource a obi:0400065 ;
            solar:hasLamp ?lamp ;
            solar:hasWavelength ?wl ;
            solar:hasIrradiance [qudt:numericValue ?irradiance] ;
            solar:hasPower [qudt:numericValue ?power] .
            
    } #LIMIT 10
""")

for row in q_res:
    print(row)
    print(f"{row.lightSource} {row.lamp} {row.wl} {row.irradiance} {row.power}")

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/7417'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('2100.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('300.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/7417 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#400-780WL 2100.0 300.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5756'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('1500.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('450.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5756 htt

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/798'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#315-400WL'), rdflib.term.Literal('32.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('32.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/798 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#315-400WL 32.5 32.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5993'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('125.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('5.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5993 http://w3id.or

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5095'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('1450.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('300.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5095 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#400-780WL 1450.0 300.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5652'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('200.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('35.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5652 http:

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/6432'), rdflib.term.URIRef('http://w3id.org/solar/o/core#OtherLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#192-280WL'), rdflib.term.Literal('59.4', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('15.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/6432 http://w3id.org/solar/o/core#OtherLamp http://w3id.org/solar/o/core#192-280WL 59.4 15.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5019'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('26.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('300.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5019 http://w3id

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/108'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#315-400WL'), rdflib.term.Literal('2250.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('150.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/108 http://w3id.org/solar/o/core#MercuryLamp http://w3id.org/solar/o/core#315-400WL 2250.0 150.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/2124'), rdflib.term.URIRef('http://w3id.org/solar/o/core#FluorescentLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('8.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('15.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/2124

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/5031'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#315-780WL'), rdflib.term.Literal('11000.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('251.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/5031 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#315-780WL 11000.0 251.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/3058'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#315-400WL'), rdflib.term.Literal('350.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('500.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/3058 ht

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/6904'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('1000.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('300.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/6904 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#400-780WL 1000.0 300.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/2648'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#192-280WL'), rdflib.term.Literal('1500.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('200.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/2648 h

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/4'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#192-280WL'), rdflib.term.Literal('8.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('8.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/4 http://w3id.org/solar/o/core#MercuryLamp http://w3id.org/solar/o/core#192-280WL 8.0 8.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/4585'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#315-780WL'), rdflib.term.Literal('800.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('300.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/4585 http://w3id.org/

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/3043'), rdflib.term.URIRef('http://w3id.org/solar/o/core#XenonLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('350.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('500.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/3043 http://w3id.org/solar/o/core#XenonLamp http://w3id.org/solar/o/core#400-780WL 350.0 500.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/373'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#400-780WL'), rdflib.term.Literal('110.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('60.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/373 http://

(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/7474'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#192-780WL'), rdflib.term.Literal('440.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('100.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/7474 http://w3id.org/solar/o/core#MercuryLamp http://w3id.org/solar/o/core#192-780WL 440.0 100.0
(rdflib.term.URIRef('http://w3id.org/solar/i/Condition/LightSource/3871'), rdflib.term.URIRef('http://w3id.org/solar/o/core#MercuryLamp'), rdflib.term.URIRef('http://w3id.org/solar/o/core#192-280WL'), rdflib.term.Literal('32.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')), rdflib.term.Literal('250.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
http://w3id.org/solar/i/Condition/LightSource/3871 h

### Inputs

In [14]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?input) AS ?count)
    WHERE {
        ?input a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Catalyst ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            solar:depositionMethod ?dmethod ;
            solar:hasBandGap ?bgap ;
            solar:hasSurfaceArea ?surfacearea .
    }

""")

for row in q_res:
    print(f"Number of catalyts with all properties: {row['count'].value}")

Number of catalyts with all properties: 1445


In [15]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?cocatalyst) AS ?count) 
    WHERE {
        ?cocatalyst a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Co-catalyst ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of co-catalyts with all properties: {row['count'].value}")

Number of co-catalyts with all properties: 3176


In [16]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?support) AS ?count) 
    WHERE {
        ?support a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solarpc:Support ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of supports with all properties: {row['count'].value}")

Number of supports with all properties: 174


In [17]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?dopant) AS ?count) 
    WHERE {
        ?dopant a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solar:Dopant ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of dopants with all properties: {row['count'].value}")

Number of dopants with all properties: 541


In [18]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>

    SELECT (COUNT (DISTINCT ?dye) AS ?count) 
    WHERE {
        ?dye a solar:Input ;
            rdfs:label ?label ;
            solar:hasRole solarpc:Dye ;
            qudt:floatPercentage ?percent .
    }

""")

for row in q_res:
    print(f"Number of dyes with all properties: {row['count'].value}")

Number of dyes with all properties: 33


## Experiment outputs generation

In [19]:
!python -m mapeathor -i mappings/outputs-solarchem-mapping.xlsx -l yarrrml -o mappings/outputs-solarchem-mapping.yml

Generating mapping file
Your mapping file is in mappings/outputs-solarchem-mapping.yml


In [20]:
!python -m morph_kgc config-outputs.ini

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:390: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fnml_df = fnml_df.applymap(str)
INFO | 2024-03-27 17:30:31,006 | 265 mapping rules retrieved.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_parser.py:485: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.rml_df = self.rml_df.replace(r'^\s*$', np.nan, regex=True)
INFO | 2024-03-27 17:30:31,312 | Mapping partition with 255 groups generated.
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/mapping/mapping_partitioner.py:182: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In 

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/utils.py:257: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace(config.get_na_values(), np.NaN)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map ins

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/utils.py:257: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replac

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  d

/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
/Users/aiglesias/miniconda3/envs/kgc/lib/python3.9/site-packages/morph_kgc/materializer.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(str)
INFO | 2024-03-27 17:31:03,446 | Number of triples generated in total: 1377484.
INFO | 2024-03-27 17:31:03,446 | Materialization finished in 32.101 seconds.


In [21]:
g = rdflib.Graph()
g.parse("result/outputs-solarchem.nt")
print(len(g))

1377484


In [22]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT ?output ?label ?value ?unit ?expexec
    WHERE {
        ?output a solar:Output ;
            rdfs:label ?label ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"{row.output} {row.label} {row.value} {row.unit} {row.expexec}")

http://w3id.org/solar/i/Output/C4H10_mol_g/785 C4H10 0.63 http://qudt.org/vocab/unit/MicroMOL-PER-GM http://w3id.org/solar/i/ExpExec/785
http://w3id.org/solar/i/Output/C4H10_mol_g/786 C4H10 0.17 http://qudt.org/vocab/unit/MicroMOL-PER-GM http://w3id.org/solar/i/ExpExec/786
http://w3id.org/solar/i/Output/C4H10_mol_g/787 C4H10 0.53 http://qudt.org/vocab/unit/MicroMOL-PER-GM http://w3id.org/solar/i/ExpExec/787
http://w3id.org/solar/i/Output/C4H10_mol_g/784 C4H10 0.57 http://qudt.org/vocab/unit/MicroMOL-PER-GM http://w3id.org/solar/i/ExpExec/784
http://w3id.org/solar/i/Output/C2H6_mol_gh/2421 C2H6 2.18 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2421
http://w3id.org/solar/i/Output/C2H6_mol_gh/3590 C2H6 0.95 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3590
http://w3id.org/solar/i/Output/C2H6_mol_gh/1837 C2H6 0.8 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/1837
http://w3id.org/solar/i/Output

http://w3id.org/solar/i/Output/C3H6_mol_m2h/2428 C3H6 0.41 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2428
http://w3id.org/solar/i/Output/C3H6_mol_m2h/2803 C3H6 0.14 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2803
http://w3id.org/solar/i/Output/C3H6_mol_m2h/2800 C3H6 0.01 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2800
http://w3id.org/solar/i/Output/C3H6_mol_m2h/2987 C3H6 0.05 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2987
http://w3id.org/solar/i/Output/C3H6_mol_m2h/912 C3H6 641.64 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/912
http://w3id.org/solar/i/Output/C3H6_mol_m2h/2421 C3H6 0.02 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2421
http://w3id.org/solar/i/Output/C3H6_mol_m2h/2985 C3H6 0.01 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2985
http:/

http://w3id.org/solar/i/Output/C2H4O2_mol_gh/6206 C2H4O2 0.29 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/6206
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/1937 C2H4O2 0.04 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/1937
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/242 C2H4O2 178.87 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/242
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/845 C2H4O2 63.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/845
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/6207 C2H4O2 1.08 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/6207
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/843 C2H4O2 54.8 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/843
http://w3id.org/solar/i/Output/C2H4O2_mol_gh/2630 C2H4O2 53.42 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/E

http://w3id.org/solar/i/Output/C3H8_mol_gh/2424 C3H8 0.32 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2424
http://w3id.org/solar/i/Output/C3H8_mol_gh/2426 C3H8 3.71 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2426
http://w3id.org/solar/i/Output/C3H8_mol_gh/787 C3H8 0.04 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/787
http://w3id.org/solar/i/Output/C3H8_mol_gh/2422 C3H8 0.99 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2422
http://w3id.org/solar/i/Output/C3H8_mol_gh/2427 C3H8 8.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2427
http://w3id.org/solar/i/Output/C3H8_mol_gh/2986 C3H8 1.12 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2986
http://w3id.org/solar/i/Output/C3H8_mol_gh/1645 C3H8 250.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/1645
http://w3id.org

http://w3id.org/solar/i/Output/C3H6_mol_gh/3592 C3H6 0.36 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3592
http://w3id.org/solar/i/Output/C3H6_mol_gh/2420 C3H6 2.31 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2420
http://w3id.org/solar/i/Output/C3H6_mol_gh/6800 C3H6 1.37 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/6800
http://w3id.org/solar/i/Output/C3H6_mol_gh/2423 C3H6 0.78 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2423
http://w3id.org/solar/i/Output/C3H6_mol_gh/2798 C3H6 0.01 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2798
http://w3id.org/solar/i/Output/C3H6_mol_gh/2986 C3H6 0.23 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2986
http://w3id.org/solar/i/Output/C3H6_mol_gh/912 C3H6 3.86 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/912
http://w3id.org

...

http://w3id.org/solar/i/Output/H2_mol_gh/7111 H2 4.5 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/7111
http://w3id.org/solar/i/Output/H2_mol_gh/5819 H2 15000.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/5819
http://w3id.org/solar/i/Output/H2_mol_gh/2565 H2 0.06 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2565
http://w3id.org/solar/i/Output/H2_mol_gh/5500 H2 300.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/5500
http://w3id.org/solar/i/Output/H2_mol_gh/4503 H2 44.44 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/4503
http://w3id.org/solar/i/Output/H2_mol_gh/1 H2 3.75 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/1
http://w3id.org/solar/i/Output/H2_mol_gh/2434 H2 163.33 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2434
http://w3id.org/solar/i/Output/H2_mol_gh/

http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2801 C2H5OH 0.52 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2801
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2755 C2H5OH 0.4 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2755
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/182 C2H5OH 55.29 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/182
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2797 C2H5OH 2.21 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2797
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2802 C2H5OH 5.48 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2802
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2795 C2H5OH 0.86 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/solar/i/ExpExec/2795
http://w3id.org/solar/i/Output/C2H5OH_mol_m2h/2756 C2H5OH 0.42 http://qudt.org/vocab/unit/MicroMOL-PER-M2-HR http://w3id.org/s

http://w3id.org/solar/i/Output/HCOOH_mol_gh/190 HCOOH 7.25 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/190
http://w3id.org/solar/i/Output/HCOOH_mol_gh/191 HCOOH 1.28 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/191
http://w3id.org/solar/i/Output/HCOOH_mol_gh/6379 HCOOH 2.35 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/6379
http://w3id.org/solar/i/Output/HCOOH_mol_gh/4720 HCOOH 6980.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/4720
http://w3id.org/solar/i/Output/HCOOH_mol_gh/2363 HCOOH 0.37 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/2363
http://w3id.org/solar/i/Output/HCOOH_mol_gh/4976 HCOOH 7.19 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/4976
http://w3id.org/solar/i/Output/HCOOH_mol_gh/3651 HCOOH 1300.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3651

http://w3id.org/solar/i/Output/C2H5OH_mol_gh/554 C2H5OH 62.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/554
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/5392 C2H5OH 140.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/5392
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/3776 C2H5OH 60.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3776
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/5076 C2H5OH 4.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/5076
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/401 C2H5OH 1.6 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/401
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/400 C2H5OH 2.6 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/400
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/170 C2H5OH 103.38 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpEx

http://w3id.org/solar/i/Output/C2H5OH_mol_gh/858 C2H5OH 1.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/858
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/7648 C2H5OH 4.3 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/7648
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/3769 C2H5OH 47.25 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3769
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/3764 C2H5OH 47.25 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3764
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/6292 C2H5OH 24.9 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/6292
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/3777 C2H5OH 79.13 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/ExpExec/3777
http://w3id.org/solar/i/Output/C2H5OH_mol_gh/5077 C2H5OH 3.0 http://qudt.org/vocab/unit/MicroMOL-PER-GM-HR http://w3id.org/solar/i/

In [24]:
q_res = g.query("""    
    PREFIX bibo: <http://purl.org/ontology/bibo/>
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX solar: <http://w3id.org/solar/o/core#>
    PREFIX solarpc: <http://w3id.org/solar/o/pc#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX qudt: <http://qudt.org/2.1/schema/qudt>
    PREFIX prov: <http://www.w3.org/ns/prov#>

    SELECT (COUNT (DISTINCT ?output) AS ?count)
    WHERE {
        ?output a solar:Output ;
            rdfs:label ?label ;
            qudt:numericValue ?value ;
            qudt:unit ?unit ;
            prov:wasGeneratedBy ?expexec .
    }

""")

for row in q_res:
    print(f"Number of outputs with all properties: {row['count'].value}")

Number of outputs with all properties: 18028


## Merging KGs into one

In [ ]:
!cat result/solarchem.nt result/outputs-solarchem.nt | uniq > result/solarchem-kg.nt